# Final Project on Image Processing class
#### Done by Bolat Tleubayev and Zhanel Zhexenova

Library import

In [1]:
import cv2
import numpy as np
#used to improve perfomance of functions
import numba
from numba import jit
import matplotlib.pyplot as plt

Functions definition

In [2]:
#erosion function
@jit('i4[:,:](f8[:,:],f8[:,:])')
def binary_erosion(input_array, structure=np.ones((3,3)).astype(np.int64)):
    
    padding = np.int64(np.floor(np.shape(structure)[0]/2))
    
    rows, cols = np.shape(input_array)
    
    padded_img = np.zeros(( np.int64(2*padding+rows), np.int64(2*padding+cols) ))
    padded_img[padding:rows+padding,padding:cols+padding] = input_array 
    
    binary_erosion = np.zeros(np.shape(padded_img))
    
    for row in range(padding, padding+rows):
        for col in range(padding, padding+cols):
            binary_erosion[row,col] = np.min(padded_img[(row-padding):(row+padding+1),
                                                        (col-padding):(col+padding+1)])
            
    out = binary_erosion[padding:rows+padding,padding:cols+padding]
                
    return out

#dilation function
@jit('i4[:,:](f8[:,:],f8[:,:])')
def binary_dilation(input_array, structure=np.ones((3,3)).astype(np.int64)):
    
    padding = np.int64(np.floor(np.shape(structure)[0]/2))
    
    rows, cols = np.shape(input_array)
    
    padded_img = np.zeros(( np.int64(2*padding+rows), np.int64(2*padding+cols) ))
    padded_img[padding:rows+padding,padding:cols+padding] = input_array 
    
    binary_dilation = np.zeros(np.shape(padded_img))
    
    for row in range(padding, padding+rows):
        for col in range(padding, padding+cols):
            binary_dilation[row,col] = np.max(padded_img[(row-padding):(row+padding+1),
                                                        (col-padding):(col+padding+1)])
            
    out = binary_dilation[padding:rows+padding,padding:cols+padding]
                
    return out

#helper thresholding function
@jit('i4(f8,f8,f8)')
def thresholder(pixel, a, b):
    if pixel > a and pixel < b:
        return 255
    else:
        return 0

#RGB thresholding function
@jit('i4[:,:](f8[:,:])')
def rgb_thr(image):
    #image = image*255
    out = np.zeros((np.shape(image)[0],np.shape(image)[1]))
    for i in range(np.shape(image[:,:,1])[0]):
        for j in range(np.shape(image[:,:,1])[1]):        
            if thresholder(image[i,j,0], 120, 240)==255 and thresholder(image[i,j,1], 90, 230)==255 and thresholder(image[i,j,2], 100, 230)==255:
                out[i,j] = 255
            else:
                out[i,j] = 0
    return out

#Moore's boundary tracing algorithm
def beetle(bimg, limg, y,x, yc, xc):
    #c position 1
    if yc == y and xc < x:
        if bimg[y,x-1] == 255:#
            by = y
            bx = x - 1
            cy = y + 1
            cx = x - 1
        elif bimg[y-1,x-1] == 255:#
            by = y - 1
            bx = x - 1
            cy = y
            cx = x - 1
        elif bimg[y-1,x] == 255:#
            by = y - 1
            bx = x
            cy = y - 1
            cx = x - 1
        elif bimg[y-1,x+1] == 255:#
            by = y - 1
            bx = x + 1
            cy = y - 1
            cx = x
        elif bimg[y,x+1] == 255:#
            by = y
            bx = x + 1
            cy = y - 1
            cx = x + 1
        elif bimg[y+1,x+1] == 255:#
            by = y + 1
            bx = x + 1
            cy = y
            cx = x + 1
        elif bimg[y+1,x] == 255:#
            by = y + 1
            bx = x
            cy = y + 1
            cx = x + 1
        elif bimg[y+1,x-1] == 255:
            by = y + 1
            bx = x - 1
            cy = y + 1
            cx = x
        else:
            bx = -1
            by = -1
            cx = -1
            cy = -1
            print("error")
    #c position 2
    if yc < y and xc < x:
        if bimg[y-1,x-1] == 255:#
            by = y - 1
            bx = x - 1
            cy = y
            cx = x - 1
        elif bimg[y-1,x] == 255:#
            by = y - 1
            bx = x
            cy = y - 1
            cx = x - 1
        elif bimg[y-1,x+1] == 255:#
            by = y - 1
            bx = x + 1
            cy = y - 1
            cx = x
        elif bimg[y,x+1] == 255:#
            by = y
            bx = x + 1
            cy = y - 1
            cx = x + 1
        elif bimg[y+1,x+1] == 255:#
            by = y + 1
            bx = x + 1
            cy = y
            cx = x + 1
        elif bimg[y+1,x] == 255:#
            by = y + 1
            bx = x
            cy = y + 1
            cx = x + 1
        elif bimg[y+1,x-1] == 255:
            by = y + 1
            bx = x - 1
            cy = y + 1
            cx = x
        elif bimg[y,x-1] == 255:#
            by = y
            bx = x - 1
            cy = y + 1
            cx = x - 1
        else:
            bx = -1
            by = -1
            cx = -1
            cy = -1
            print("error")
    #c position 3
    if yc < y and xc == x:
        if bimg[y-1,x] == 255:#
            by = y - 1
            bx = x
            cy = y - 1
            cx = x - 1
        elif bimg[y-1,x+1] == 255:#
            by = y - 1
            bx = x + 1
            cy = y - 1
            cx = x
        elif bimg[y,x+1] == 255:#
            by = y
            bx = x + 1
            cy = y - 1
            cx = x + 1
        elif bimg[y+1,x+1] == 255:#
            by = y + 1
            bx = x + 1
            cy = y
            cx = x + 1
        elif bimg[y+1,x] == 255:#
            by = y + 1
            bx = x
            cy = y + 1
            cx = x + 1
        elif bimg[y+1,x-1] == 255:
            by = y + 1
            bx = x - 1
            cy = y + 1
            cx = x
        elif bimg[y,x-1] == 255:#
            by = y
            bx = x - 1
            cy = y + 1
            cx = x - 1
        elif bimg[y-1,x-1] == 255:#
            by = y - 1
            bx = x - 1
            cy = y
            cx = x - 1
        else:
            bx = -1
            by = -1
            cx = -1
            cy = -1
            print("error")
    #c position 4
    if yc < y and xc > x:
        if bimg[y-1,x+1] == 255:#
            by = y - 1
            bx = x + 1
            cy = y - 1
            cx = x
        elif bimg[y,x+1] == 255:#
            by = y
            bx = x + 1
            cy = y - 1
            cx = x + 1
        elif bimg[y+1,x+1] == 255:#
            by = y + 1
            bx = x + 1
            cy = y
            cx = x + 1
        elif bimg[y+1,x] == 255:#
            by = y + 1
            bx = x
            cy = y + 1
            cx = x + 1
        elif bimg[y+1,x-1] == 255:
            by = y + 1
            bx = x - 1
            cy = y + 1
            cx = x
        elif bimg[y,x-1] == 255:#
            by = y
            bx = x - 1
            cy = y + 1
            cx = x - 1
        elif bimg[y-1,x-1] == 255:#
            by = y - 1
            bx = x - 1
            cy = y
            cx = x - 1
        elif bimg[y-1,x] == 255:#
            by = y - 1
            bx = x
            cy = y - 1
            cx = x - 1
        else:
            bx = -1
            by = -1
            cx = -1
            cy = -1
            print("error") 
    #c position 5
    if yc == y and xc > x:
        if bimg[y,x+1] == 255:#
            by = y
            bx = x + 1
            cy = y - 1
            cx = x + 1
        elif bimg[y+1,x+1] == 255:#
            by = y + 1
            bx = x + 1
            cy = y
            cx = x + 1
        elif bimg[y+1,x] == 255:#
            by = y + 1
            bx = x
            cy = y + 1
            cx = x + 1
        elif bimg[y+1,x-1] == 255:
            by = y + 1
            bx = x - 1
            cy = y + 1
            cx = x
        elif bimg[y,x-1] == 255:#
            by = y
            bx = x - 1
            cy = y + 1
            cx = x - 1
        elif bimg[y-1,x-1] == 255:#
            by = y - 1
            bx = x - 1
            cy = y
            cx = x - 1
        elif bimg[y-1,x] == 255:#
            by = y - 1
            bx = x
            cy = y - 1
            cx = x - 1
        elif bimg[y-1,x+1] == 255:#
            by = y - 1
            bx = x + 1
            cy = y - 1
            cx = x
        else:
            bx = -1
            by = -1
            cx = -1
            cy = -1
            print("error") 
    #c position 6
    if yc > y and xc > x:
        if bimg[y+1,x+1] == 255:#
            by = y + 1
            bx = x + 1
            cy = y
            cx = x + 1
        elif bimg[y+1,x] == 255:#
            by = y + 1
            bx = x
            cy = y + 1
            cx = x + 1
        elif bimg[y+1,x-1] == 255:
            by = y + 1
            bx = x - 1
            cy = y + 1
            cx = x
        elif bimg[y,x-1] == 255:#
            by = y
            bx = x - 1
            cy = y + 1
            cx = x - 1
        elif bimg[y-1,x-1] == 255:#
            by = y - 1
            bx = x - 1
            cy = y
            cx = x - 1
        elif bimg[y-1,x] == 255:#
            by = y - 1
            bx = x
            cy = y - 1
            cx = x - 1
        elif bimg[y-1,x+1] == 255:#
            by = y - 1
            bx = x + 1
            cy = y - 1
            cx = x
        elif bimg[y,x+1] == 255:#
            by = y
            bx = x + 1
            cy = y - 1
            cx = x + 1
        else:
            bx = -1
            by = -1
            cx = -1
            cy = -1
            print("error")         
    #c position 7
    if yc > y and xc == x:
        if bimg[y+1,x] == 255:#
            by = y + 1
            bx = x
            cy = y + 1
            cx = x + 1
        elif bimg[y+1,x-1] == 255:
            by = y + 1
            bx = x - 1
            cy = y + 1
            cx = x
        elif bimg[y,x-1] == 255:#
            by = y
            bx = x - 1
            cy = y + 1
            cx = x - 1
        elif bimg[y-1,x-1] == 255:#
            by = y - 1
            bx = x - 1
            cy = y
            cx = x - 1
        elif bimg[y-1,x] == 255:#
            by = y - 1
            bx = x
            cy = y - 1
            cx = x - 1
        elif bimg[y-1,x+1] == 255:#
            by = y - 1
            bx = x + 1
            cy = y - 1
            cx = x
        elif bimg[y,x+1] == 255:#
            by = y
            bx = x + 1
            cy = y - 1
            cx = x + 1
        elif bimg[y+1,x+1] == 255:#
            by = y + 1
            bx = x + 1
            cy = y
            cx = x + 1
        else:
            bx = -1
            by = -1
            cx = -1
            cy = -1
            print("error")
    #c position 8
    if yc > y and xc < x:
        if bimg[y+1,x-1] == 255:
            by = y + 1
            bx = x - 1
            cy = y + 1
            cx = x
        elif bimg[y,x-1] == 255:#
            by = y
            bx = x - 1
            cy = y + 1
            cx = x - 1
        elif bimg[y-1,x-1] == 255:#
            by = y - 1
            bx = x - 1
            cy = y
            cx = x - 1
        elif bimg[y-1,x] == 255:#
            by = y - 1
            bx = x
            cy = y - 1
            cx = x - 1
        elif bimg[y-1,x+1] == 255:#
            by = y - 1
            bx = x + 1
            cy = y - 1
            cx = x
        elif bimg[y,x+1] == 255:#
            by = y
            bx = x + 1
            cy = y - 1
            cx = x + 1
        elif bimg[y+1,x+1] == 255:#
            by = y + 1
            bx = x + 1
            cy = y
            cx = x + 1
        elif bimg[y+1,x] == 255:#
            by = y + 1
            bx = x
            cy = y + 1
            cx = x + 1
        else:
            bx = -1
            by = -1
            cx = -1
            cy = -1
            print("error")
    
    return by, bx, cy, cx
    
#looking at 4 neighbours, if there are white pixels in all of them
#then it is not a boundary pixel
@jit('i4(f8[:,:],i4,i4)')
def hood(bimg, y, x):
    if bimg[y,x-1] == 255 and bimg[y-1,x] == 255 and bimg[y,x+1] == 255 and bimg[y+1,x] == 255:
        hoodType = 1
    else:
        hoodType = 0
    return hoodType

#function to extract contour data
@jit('f8[:,:](f8[:,:])')
def contourRetriever(binary_image):
    
    counter_cnt = 1 
    x_b = -1
    y_b = -1
    x_c = -1
    y_c = -1

    padding = 1 #shape of the checking area is 3x3, so pad 1
    rows, cols = np.shape(binary_image)
    padded_img = np.zeros(( np.int64(2*padding+rows), np.int64(2*padding+cols) ))
    padded_img[padding:rows+padding,padding:cols+padding] = binary_image 
    padded_lab = -1*np.ones(np.shape(padded_img))

    contours_ret = []
    cnt_cols = []

    for i in range(padding, padding+rows):
        for j in range(padding, padding+cols):
            if padded_lab[i,j] == -1 and padded_img[i,j] > 120:
                y_b = i
                x_b = j
                y_c = i 
                x_c = j - 1
                while True:
                    if padded_lab[y_c,x_c] == -1 and hood(padded_img,y_b,x_b) == 0:
                        cnt_cols.append([y_b, x_b, y_c, x_c])
                        y_b, x_b, y_c, x_c = beetle(padded_img,padded_lab, y_b,x_b, y_c, x_c)
                        padded_lab[y_b,x_b] = counter_cnt
                        if x_b == j and y_b == i:
                            counter_cnt = counter_cnt+1
                            contours_ret.append(cnt_cols)
                            cnt_cols = []
                            break
                    else:
                        break
    return contours_ret
#homography estimation function
@jit('f8[:,:](f8,f8,f8,f8)')
def homography (x_old,y_old,x_new, y_new):

    
    A = np.array([[x_old[0], y_old[0], 1, 0, 0, 0, -x_old[0]*x_new[0], -y_old[0]*x_new[0]],
         [0, 0, 0, x_old[0], y_old[0], 1, -x_old[0]*y_new[0], -y_old[0]*y_new[0]], 
         [x_old[1], y_old[1], 1, 0, 0, 0, -x_old[1]*x_new[1], -y_old[1]*x_new[1]],
         [0, 0, 0, x_old[1], y_old[1], 1, -x_old[1]*y_new[1], -y_old[1]*y_new[1]],
         [x_old[2], y_old[2], 1, 0, 0, 0, -x_old[2]*x_new[2], -y_old[2]*x_new[2]],
         [0, 0, 0, x_old[2], y_old[2], 1, -x_old[2]*y_new[2], -y_old[2]*y_new[2]],
         [x_old[3], y_old[3], 1, 0, 0, 0, -x_old[3]*x_new[3], -y_old[3]*x_new[3]],
         [0, 0, 0, x_old[3], y_old[3], 1, -x_old[3]*y_new[3], -y_old[3]*y_new[3]]])
    
    b = [x_new[0],y_new[0],x_new[1],y_new[1],x_new[2],y_new[2],x_new[3],y_new[3]]
    
    if np.linalg.det(A) != 0:
        h = np.linalg.solve(A,np.transpose(b))
    else:
        u, s, vh = np.linalg.svd(A, full_matrices=True)
        h = vh[:,7]
        
    return h
#pixel location extraction from homography
def pixLocation(h, x, y):
    x_1 = (h[0]*x + h[1]*y + h[2])/(h[6]*x+h[7]*y+1);
    y_1 = (h[3]*x + h[4]*y + h[5])/(h[6]*x+h[7]*y+1);
    return x_1,y_1

@jit('f8[:,:,:](f8[:,:,:],i4,i4)')
def resize(inp, newSizeX,newSizeY):
    oldSizeX = np.shape(inp)[1]
    oldSizeY = np.shape(inp)[0]
    #newSizeX = int(np.round(oldSizeX*scaleX))
    scaleX = newSizeX/oldSizeX
    scaleY = newSizeY/oldSizeY
    #newSizeY = int(np.round(oldSizeY*scaleY))
    if len(np.shape(inp)) == 3 :
        output =  np.zeros([newSizeY, newSizeX, 3], dtype=np.uint8)
    elif len(np.shape(inp)) == 2 :  
        output =  np.zeros([newSizeY, newSizeX], dtype=np.uint8)
    else:
        print('input is not an image')    
          
             
    for x in range(0,newSizeX-1):
        for y in range (0,newSizeY-1):
            oldX = int(np.round(x/scaleX))
            oldY = int(np.round(y/scaleY))
            
            if  (oldX > oldSizeX) or (oldY > oldSizeY):
                output[y,x] = 0
            else:                  
                if len(np.shape(inp)) == 3:                 
                    for k in range (0,2):
                        output[y,x,k] = inp[oldY,oldX,k]
                else:
                    output[y,x] = inp[oldY,oldX] 
             
                    
    return output          


Variables initialization

In [3]:
#reading the video
cap = cv2.VideoCapture('video.mp4')
#initializing variables for future use
#masks for morphological operations
kernel = np.ones((3,3),np.uint8)
kernel2 = np.ones((7,7),np.uint8)
#scaling video for better perfoance
diminisher = 3 #3 works better than 2
width = np.int64(1280/diminisher)
height = np.int64(720/diminisher)
#text parameters
font                   = cv2.FONT_HERSHEY_SIMPLEX
#bottomLeftCornerOfText = (10,500)
fontScale              = 1
fontColor              = (100,255,255)
lineType               = 2
helper1 = cv2.imread('helper2.jpg')
helper2 = cv2.imread('helper3.jpg')
helper1 = cv2.cvtColor(helper1, cv2.COLOR_BGR2GRAY)
helper2 = cv2.cvtColor(helper2, cv2.COLOR_BGR2GRAY)
bg = cv2.imread('clear.jpg')
val = 0
indexList = []
prevIndexList = []
car_id = 0
counter = 0
detectionSize = np.int64(30/diminisher)
contourL = 45/diminisher
#homography estimation values
homogX = 0
homogY = 0
homogX1 = 0
homogY1 = 0
homogX2 = 0
homogY2 = 0
speed = 0
distM = 0
#x_old = [ 374.25, 440.25, 600.75, 1004.25]
#y_old = [ 125.75, 118.25, 686.75, 544.25]
#x_new = [374.25, 440.25, 374.25, 440.25]
#y_new = [125.75, 125.75, 686.75, 686.75]

x_old = [ 126, 146, 214, 378]
y_old = [ 44, 40, 236, 195]
x_new = [126, 146, 126, 146]
y_new = [44, 44, 236, 236]
#getting homography
homo = homography (x_old,y_old,x_new, y_new)

oneoverfps=0.033

The main loop

In [39]:

while (1):
#for lup in range(30):
    
    #Reading frames
    _, frame = cap.read()
    
    #Resizing
   # frame =  resize(frame, width, height) 
    frame =  cv2.resize(frame, (width, height)) 
    
    #collecting the first frame for tracking
    if counter == 0:
        prevFrame = frame
    #background image
    #bg =  resize(bg, width, height) 
    bg =  cv2.resize(bg, (width, height)) 
    #ROI images
    helper1 =  cv2.resize(helper1, (width, height))
    helper2 =  cv2.resize(helper2, (width, height))
    #helper1 =  resize(helper1, width, height)
    #helper2 =  resize(helper2, width, height)
    
    #RGB-------------------------------------------
    
    mask = rgb_thr(frame)
    thr = 220
    lol = (helper1+helper2) - mask
    lol[lol>=thr] = 255
    lol[lol<thr] = 0
    erosion = binary_erosion(lol,kernel2)
    dilation = binary_dilation(erosion,kernel2)
    dilation = binary_dilation(dilation,kernel2)
    dilation = binary_dilation(dilation,kernel)
    dilation = binary_dilation(dilation,kernel)
    erosion = binary_erosion(dilation,kernel)
    dilation = binary_dilation(erosion,kernel)

    #----------------------------------------------
    #implementaton of contour finding
    contours = contourRetriever(dilation)
    
    for i in range(np.shape(contours)[0]):
        
        contourThreshold = contourL
        
        if len(contours[i]) >= np.int64(contourThreshold):
            cnt=contours[i] 
            y = np.mean(np.array(contours[i])[:,0])
            x = np.mean(np.array(contours[i])[:,1])
          
        
            if counter == 0:
                prevIndexList.append([np.int64(x),np.int64(y),car_id, 0])
                car_id = car_id+1 
                
            else:
                indexList.append([np.int64(x),np.int64(y),car_id, 0])
                
                car_id = car_id+1
                #print("f",str(counter),":",[np.int64(x+w/2),np.int64(y+h/2),w,h,car_id])
                
    if counter != 0:
        for i in range(np.shape(prevIndexList)[0]):
            xi = prevIndexList[i][0]
            yi = prevIndexList[i][1]
            ii = prevIndexList[i][2]
            
            
            for j in range(np.shape(indexList)[0]):
            
                xj = indexList[j][0]
                yj = indexList[j][1]
                ij = indexList[j][2]
                if pixLocation(homo, xi,yi)[0] > 0:
                    homogX1  = pixLocation(homo, xi,yi)[0] 
                else:
                    homogX1 = 0
                if pixLocation(homo, xi,yi)[1] > 0:
                    homogY1  = pixLocation(homo, xi,yi)[1] 
                else:
                    homogY1 = 0    
                if pixLocation(homo, xj,yj)[0] > 0:
                    homogX2  = pixLocation(homo, xj,yj)[0] 
                else:
                    homogX2 = 0
                if pixLocation(homo, xj,yj)[1] > 0:
                    homogY2  = pixLocation(homo, xj,yj)[1] 
                else:
                    homogY2 = 0 
                
                distM = np.sqrt((homogX1-homogX2)**2 + (homogY1-homogY2)**2)*59.82/(height)/0.05
                dist = np.sqrt((xi-xj)**2 + (yi-yj)**2)
                if(dist < detectionSize) and (np.abs(xi-xj)<12) and (np.abs(yi-yj)<12):
                    indexList[j][2] = ii
                    indexList[j][3] = int(np.round(distM))
                     
                    break
                 
           
                
            #(x,y) coordinates of top left corner of rectangle
            #(w,h) width and height of the rectangle
            
        for i in range(np.shape(indexList)[0]):
           # if indexList[i][3] == 0 and ((np.shape(prevIndexList)[0]) == 0 or (np.sum(prevIndexList, axis = 0)[3]) == 0):
            #    speed = str(4)
            if  indexList[i][3] > 15:  #np.shape(prevIndexList)[0] == 0 
                if int(np.round(np.sum(prevIndexList, axis = 0)[3]/(np.shape(prevIndexList)[0])))>=3 and int(np.round(np.sum(prevIndexList, axis = 0)[3]/(np.shape(prevIndexList)[0])))<=15:
                    speed =  str(int(np.round(np.sum(prevIndexList, axis = 0)[3]/(np.shape(prevIndexList)[0]))))
            elif indexList[i][3] < 3:
                
                if np.shape(prevIndexList)[0] == 0:   
                    speed = str(3)
                elif int(np.round(np.sum(prevIndexList, axis = 0)[3]/(np.shape(prevIndexList)[0])))>= 3 and int(np.round(np.sum(prevIndexList, axis = 0)[3]/(np.shape(prevIndexList)[0])))<=15:   
                    speed = str(int(np.round(np.sum(prevIndexList, axis = 0)[3]/(np.shape(prevIndexList)[0]))))
                elif int(np.round(np.sum(prevIndexList, axis = 0)[3]/(np.shape(prevIndexList)[0])))>=15:
                    speed = str(int((indexList[i][3]+np.round(np.sum(prevIndexList, axis = 0)[3]/(np.shape(prevIndexList)[0])))/2))                       
                elif np.sum(prevIndexList, axis = 0)[3]/np.shape(prevIndexList)[0]< 3:
                    speed =str(3) 
               
            else:
                speed = str(indexList[i][3])
                
            dilation = cv2.circle(frame,(np.int64(indexList[i][0]),np.int64(indexList[i][1])),detectionSize,(0,255,0),2)
            #coordinates text
            dilation = cv2.putText(frame,"id:"+str(indexList[i][2])+" speed:"+speed,
                                   (np.int64(indexList[i][0]),np.int64(indexList[i][1])),
                                   font, 
                                   fontScale/2,
                                   fontColor,
                                   lineType)
            #frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(155,0,0),2)
         #   print("heere ", np.sum(prevIndexList, axis = 0)[3],"shpe: ",np.shape(prevIndexList)[0],"dist: ",distM)
            print("curr ID: ",indexList[i][2],"prev ID: ",prevIndexList[:][2])
    #if counter < 100:
    #    cv2.imwrite(str(counter)+".jpg", frame)
    
    prevIndexList = indexList
    indexList = []
    
    cv2.imshow('aaa', frame)    
    
    prevFrame = frame
    
    #counting loops of while
    counter = counter+1
    
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
cv2.destroyAllWindows()

curr ID:  1537 prev ID:  [360, 180, 1590, 2]
curr ID:  1551 prev ID:  [360, 180, 1590, 2]
curr ID:  1590 prev ID:  [360, 180, 1590, 2]
curr ID:  1615 prev ID:  [360, 180, 1590, 2]
curr ID:  1537 prev ID:  [356, 180, 1590, 2]
curr ID:  1551 prev ID:  [356, 180, 1590, 2]
curr ID:  1590 prev ID:  [356, 180, 1590, 2]
curr ID:  1615 prev ID:  [356, 180, 1590, 2]
curr ID:  1537 prev ID:  [350, 179, 1590, 3]
curr ID:  1551 prev ID:  [350, 179, 1590, 3]
curr ID:  1590 prev ID:  [350, 179, 1590, 3]
curr ID:  1615 prev ID:  [350, 179, 1590, 3]
curr ID:  1537 prev ID:  [347, 178, 1590, 1]
curr ID:  1551 prev ID:  [347, 178, 1590, 1]
curr ID:  1590 prev ID:  [347, 178, 1590, 1]
curr ID:  1615 prev ID:  [347, 178, 1590, 1]
curr ID:  1537 prev ID:  [343, 177, 1590, 2]
curr ID:  1551 prev ID:  [343, 177, 1590, 2]
curr ID:  1590 prev ID:  [343, 177, 1590, 2]
curr ID:  1537 prev ID:  [340, 177, 1590, 2]
curr ID:  1551 prev ID:  [340, 177, 1590, 2]
curr ID:  1590 prev ID:  [340, 177, 1590, 2]
curr ID:  

IndexError: list index out of range

In [126]:
test = []
test.append([1,2,5,6])
test.append([1,2,5,6])
test.append([1,2,5,6])

In [105]:
print(test)

[[1, 2, 5, 6], [1, 2, 5, 6], [1, 2, 5, 6]]


In [123]:
np.sum(test,axis=0)[2]

15

In [36]:
prevIndexList

[[160, 124, 1551, 1],
 [217, 122, 1537, 2],
 [360, 180, 1590, 2],
 [413, 234, 1615, 0]]

In [37]:
indexList

[]

In [38]:
int(np.round(np.sum(prevIndexList, axis = 0)[3]/(np.shape(prevIndexList)[0])))

1

In [153]:
def makingAnagrams(s1, s2):
    size1 = len(s1)
    size2 = len(s2)
    anagram = []
    same = 0
    temp = []
    s = []
    r=[]
  
    if size1>size2:
        for i in range (size2):
            for j in range(size1):
                if s2[i] == s1[j]:
                    anagram.append(s2[i])
    else:
        for i in range (size1):
            for j in range(size2):
                if s1[i] == s2[j]:
                    anagram.append(s1[i])
    temp = anagram                
    for i in range(len(anagram)):
        same = 0
        for j in range (len(anagram)):
            if anagram[i] == anagram[j]:
                same = same+1
            if same > 1:
                s.append([anagram[i],same, i])
          #  print(same)   
    leng = len(temp)
    for i in range(leng-1):
        same = 0
        t = temp.pop()
        for j in range(len(temp)):
            if t != temp[j]:
                r.insert(0,t)
            
       
    print (r,t)
    return  (size1+size2-2*len(anagram))
    

In [154]:
r = makingAnagrams("abjslvmwerfksll","bikytrrvjwknvadc")

['b', 'j', 'j', 'v', 'v', 'v', 'v', 'v', 'v', 'w', 'w', 'w', 'w', 'w', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k'] b


In [102]:
s2 =["a","b","dhfkg"]

In [103]:
for i in range (len(s1)):
    s2.append(s1[i])
    

In [112]:
#r = s2.pop()
print(s2)
s2.insert(0,"2")

['2', '2', 'a', 'b', 'dhfkg', 's', 't', 'r', 'i', 'n', '2', '2']


In [158]:
w= (str(123))

In [219]:
import sys

def Saddamian(inp,count):
    l =str(inp)
    new = 0
    isSaddamian = False
    count = count+1
    #print(count)
    for i in range(len(l)):
        new = int(l[i])**2+new
    if new!= 1 and count<100:
        return Saddamian(new,count)
    elif new ==1 :
        return ("True")
    elif new!=1 and count==100 :
        return ("False")



sad = Saddamian(7,0)
print(sad)

    

True


In [212]:
import numpy as np
def connectedCell(matrix, x, y,temp):
    
    #n = np.shape(matrix)[0]
    #m = np.shape(matrix)[1]
    count = 0
    #temp = np.zeros(n,m)
    if x == 0 and y == 0:
        if matrix[y][x] == 1:
            temp[y][x] = temp[y][x]+1
            if matrix[y][x+1] == 1: 
                temp[y][x+1] = temp[y][x]+1
                x = x+1
                y = y
            elif matrix[y+1][x] == 1: 
                temp[y+1][x] = temp[y][x]+1
                x = x
                y = y+1
            elif matrix[y+1][x+1] == 1:
                temp[y+1][x+1] = temp[y][x]+1
                x = x+1
                y = y+1
        else:
            y = y
            x = x+1
                
    elif x == 0 





if matrix[i-1][j-1] == 1:
       
            count = count+1
            temp[i][j] = 1
            temp[i-1][j-1] = temp[i][j]+1
    elif matrix[i-1][j] == 1:  
    elif matrix[i-1][j+1] == 1: 
    elif matrix[i][j-1] == 1: 
    elif matrix[i][j+1] == 1: 
    elif matrix[i+1][j-1] == 1: 
    elif matrix[i+1][j] == 1: 
    elif matrix[i+1][j+1] == 1: 
                    
                    break
            

In [215]:
a = np.array([[1,0,1,0],[1,0,1,1],[1,1,1,1]])

In [216]:
print (a)

[[1 0 1 0]
 [1 0 1 1]
 [1 1 1 1]]
